# Match Misr and PM data

In [22]:
from misr_tabular_data import MISR_tabular_data, get_misr_data_df
from pm_data import PM_Data

def check_if_pm_station_is_in_path(pm_lat,pm_lon,res,path):
    try:
        mtk.latlon_to_bls(path,res,pm_lat, pm_lon)
    except:
        return False
    return True
            


def find_matches_between_pm_to_misr_data(pm_data_file,misr_dir):
    matches_misr_pm_data = []
    pd = PM_Data(pm_data_file)
        
    pm_data = pd.load_all()
    misr_df = get_misr_data_df(misr_dir)

    res = 1100

    for pm_measure in pm_data:
        pm_lat, pm_lon = pm_measure['latitude'], pm_measure['longitude']
        pm_date = pm_measure['date_gmt']
        for path in misr_files_per_path:
            is_in_path = check_if_pm_station_is_in_path(pm_lat,pm_lon,res,path)
            if (is_in_path):
                if path not in pm_data_per_path.keys():
                    pm_data_per_path[path] = {}
                if pm_date not in pm_data_per_path[path].keys():
                    pm_data_per_path[path][pm_date] = []
                pm_data_per_path[path][pm_date].append(pm_measure)

    #return misr_files_per_path,pm_data_per_path

    for path in misr_files_per_path.keys():
         for date in misr_files_per_path[path].keys():
                if date in pm_data_per_path[path].keys():
                    for pm_station_data in pm_data_per_path[path][date]:
                        matches_misr_pm_data.append({'pm_data': pm_station_data, 'image_data' : {'file' : misr_file}})
        #                     print('Found match!')

    return matches_misr_pm_data




pm_data_file = '/home/adiraz/adiraz/misr_data/pm_la_2019.json'
misr_dir = '/home/adiraz/adiraz/misr_data'
a = find_matches_between_pm_to_misr_data(pm_data_file,misr_dir)
matches_file = '/home/adiraz/adiraz/matches_pm_misr.json'
# with open(matches_file,'w') as f:
#     json.dump(matches_misr_pm_data,f)

TypeError: string indices must be integers

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pm_data import PM_Data
from misr_tabular_data import get_misr_data_df
import pandas
import MisrToolkit as mtk
import numpy as np
pm_data_file = '/home/adiraz/adiraz/misr_data/pm_la_2019.json'
misr_dir = '/home/adiraz/adiraz/misr_data'

pdaily = PM_Data(pm_data_file)
pm_data = pdaily.load()
misr_df = get_misr_data_df(misr_dir)
pm_data['pm_pixel_location'] = np.empty((len(pm_data), 0)).tolist()
all_sites = pm_data['site_number'].unique()
paths_per_site = {}
all_misr_available_paths = misr_df['path'].unique()
resolution = 1100
for site in all_sites:
    lat = pm_data[pm_data['site_number'] == site]['latitude'].head(1)
    lon = pm_data[pm_data['site_number'] == site]['longitude'].head(1)
    all_path_data = dict.fromkeys(all_misr_available_paths)
    for path in all_misr_available_paths:
        try:
            site_pixel_location = mtk.latlon_to_bls(path, resolution, float(lat),float(lon))
            all_path_data[path] = site_pixel_location
            #print(site, path,site_pixel_location)
            #arr = np.empty(len(pm_data.loc[pm_data['site_number'] == site]), dtype=object)
            #arr[:] = [{'path': site_pixel_location for i in range(len(arr))]
            #pm_data.loc[pm_data['site_number'] == site, 'pm_pixel_location'] = arr #np.array([site_pixel_location]* len(pm_data.loc[pm_data['site_number'] == site]),dtype=object)
            #misr_df.iloc[(misr_df['path'] == path),'pm_pixel_location'].append(site_pixel_location[2])#[0],site_pixel_location[1],site_pixel_location[2])
            #misr_df.loc[(misr_df['path'] == path),'pm_pixel_location'] = [site_pixel_location]*len(all_path_misr_entries)
            #all_path_misr_entries['pm_pixel_location'] = [site_pixel_location]*len(all_path_misr_entries)
        except:
            continue
    arr = np.empty(len(pm_data.loc[pm_data['site_number'] == site]), dtype=object)
    arr[:] = [all_path_data for i in range(len(arr))]
    pm_data.loc[pm_data['site_number'] == site, 'pm_pixels_location'] = arr #np.array([site_pixel_location]* len(pm_data.loc[pm_data['site_number'] == site]),dtype=object)

    #path_list = mtk.latlon_to_path_list(lat,lon)
    #paths_per_site[site] = path_list


In [5]:
all_matches = []
def get_relevant_images(p) :
    pm_val = p['arithmetic_mean']
    date = p['date_local'] 
    path_pixels = p['pm_pixels_location']
    for path in path_pixels.keys():
        all_relevant_images = misr_df.loc[(misr_df['date'] == date) & (misr_df['path'] == path),'file_name']
        if len(all_relevant_images)> 0 :
            all_matches.append({'pm_val' : pm_val, 'images' : list(all_relevant_images)})
    return all_relevant_images

In [6]:
#TODO - downlaod pm data from 2-2019-2020 currently doesn't match/

In [7]:
pm_data.apply(get_relevant_images,axis=1)
all_matches


[{'pm_val': 5.0,
  'images': ['/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_DA_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_CA_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_BA_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_AA_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_AN_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_AF_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_BF_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_CF_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P043_O106560_DF_F03_0024.hdf']},
 {'pm_val': 3.2,
  'images': ['/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN_GM_P040_O106487_DA_F03_0024.hdf',
   '/home/adiraz/adiraz/misr_data/MISR_AM1_GRP_TERRAIN

In [ ]:
misr_df.loc[(misr_df['date'] == '2021-12-30') & (misr_df['path'] == 40)]

In [ ]:
meas = pm_data.loc[(pm_data['site_number'] == '9033') & (pm_data['date_local'] =='2021-12-30')]

In [ ]:
mtk.latlon_to_bls(40, 1100, float(meas['latitude']),float(meas['longitude']))

In [ ]:
float(meas['latitude'])

In [ ]:
all_paths = misr_df['path'].unique()

array([40, 42, 39, 41, 43])

In [6]:
all_matches

NameError: name 'all_matches' is not defined